# Capstone Main Notebook

### This Notebook will be used for the IBM Data Science Capstone Course on Coursera

## Index 
1. [Week 3 Assignment](#Week-3-Assignment)

In [23]:
import sys

!conda install --yes --prefix {sys.prefix} numpy
!conda install --yes --prefix {sys.prefix} scikit-learn
!conda install --yes --prefix {sys.prefix} pandas
!conda install --yes --prefix {sys.prefix} matplotlib

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [9]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [5]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


# Week 3 Assignment

### Part 1

In [35]:
from bs4 import BeautifulSoup
import requests
import re


In [113]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

response = requests.get(URL)
soup = BeautifulSoup(response.content, "html.parser")

raw = []
for tr in soup.tbody.find_all("tr"):
    raw.append([td.get_text().strip() for td in tr.find_all("td")])
    
data = pd.DataFrame(raw, columns = ["PostalCode", "Borough", "Neighborhood"]).drop(0, axis = 0)
data = data[data.loc[:,"Borough"] != "Not assigned"]
data["Neighborhood"] = np.where(data["Neighborhood"] == "" ,data["Borough"], data["Neighborhood"])

data.reset_index(drop = True)
data.shape

(103, 3)

### Part 2

In [114]:
import geocoder

coords = pd.read_csv("Geospatial_Coordinates.csv")
    
data = pd.merge(data, coords, left_on = "PostalCode", right_on = "Postal Code", validate = "many_to_many")

data.drop("Postal Code", axis =1, inplace = True)
data

PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A  Downtown Toronto   
..         ...               ...   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                            Regent Park, Harbourfront  43.654260 -79.360636  
3                     Lawrence Manor, Lawrence Heights  43.718518 -79.464763  
4          Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
..                                                 ...        ...        ...  
98       The Kingsway, Montgomery Road, Old Mill North  43.653654 -79.506944  
99                                Church and Wellesley  43.665860 -79.383160  
100              Business reply mail Processing Centre  43.662744 -79.321558  
101  Old Mill South, King's Mill Park, Sunnylea, Hu...  43.636258 -79.498509  
102  Mimico NW, The Queensway West, South of Bloor,...  43.628841 -79.520999  

[103 rows x 5 columns]

### Part 3

In [127]:
from sklearn.cluster import KMeans
from matplotlib import cm
from matplotlib import colors
import folium

In [153]:
cluster_points = data[["Latitude", "Longitude"]]
clusters = 7

lat = np.mean(cluster_points.iloc[:,0])
long = np.mean(cluster_points.iloc[:,1])

km = KMeans(n_clusters = clusters).fit(cluster_points)


map_clusters = folium.Map(location=[lat, long], zoom_start=11)

x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colormap = plt.cm.gist_ncar
colors_array = colormap(np.linspace(0, 1, len(ys)))
col = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(cluster_points['Latitude'], cluster_points['Longitude'], data["Neighborhood"], km.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=col[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
